In [1]:
# Retrieval Augmented Generation using Azure documentation
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader


config = dotenv_values(".env")

llm = ChatOpenAI(api_key=config["OPEN_AI_KEY"])

loader = PyPDFLoader("iotc-user-guide vs.pdf")

docs = loader.load()

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OpenAIEmbeddings(api_key=config["OPEN_AI_KEY"])

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [3]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [4]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [6]:
response = retrieval_chain.invoke({"input": "What is INFER"})
print(response["answer"])
print(type(response["answer"]))

INFER™ is an IoT & Edge management solution that improves an enterprise's operational excellence and cost efficiency. It on-boards, configures, manages, monitors, and secures unmanned IoT devices and objects at scale, enabling pre-registration and bulk onboarding of devices, managing alerts and notifications, troubleshooting, changing device configurations, viewing audit logs, and performing compliance management operations through over-the-air updates.
<class 'str'>


In [13]:
response = retrieval_chain.invoke({"input": "How do I set up my Organization on INFER?"})
print(response["answer"])


To set up your organization on INFER, you need to follow the steps outlined in the provided context. First, you must have the permissions associated with the Organization Administrator role. Then, you can create an organization by going to the INFER Console, navigating to Identity & Access > Organizations, and clicking CREATE. Follow the steps in the Basic Information step to enter the name of your organization, select the parent organization (if applicable), and enter a unique identifier. Finally, review the information you have entered and click SAVE to successfully create your organization.


In [14]:
response = retrieval_chain.invoke({"input": "How do I set up an alert on INFER?"})
print(response["answer"])


To set up an alert on INFER, you need to follow these steps:

1. Click on "OFFLINE" to trigger device offline alerts.
2. Enter the duration in minutes for the server to wait before triggering the alert.
3. Enter the action that must be taken when the alert is triggered.
4. Click "NEXT."
5. Enable notifications and select the notification to be sent when the alert is triggered.
6. Enable commands and select the commands to be executed when the alert is triggered.
7. Specify the parameters under "Argument Name" and "Argument Value."
8. Click "DONE."
9. Review the new alert definition information and click "SAVE."


In [15]:
response = retrieval_chain.invoke({"input": "Does INFER support alert notifications?"})
print(response["answer"])


Yes, INFER supports alert notifications as explained in the provided context.
